# STRATEGIA DELLE BANDE
## Un semplice modello di trading che basandosi sulle simulazioni Monte Carlo determina il prezzo futuro di un titolo finanziario e quindi ne calcola il Value-to-risk a differenti livelli.Queste vengono chiamate "bande" e il modello stabilirà se aprire una posizione long o short a seconda se il prezzo superi questi livelli.

In [1]:
#IMPOSTA I PARAMETRI

nome='BABA.csv'    # nome del file csv scaricato da yahoo finance
rolling=5          # giorno per il calcolo della deviazione standard
n_sim=500           # numero di simulazioni da eseguire
perc=[0.95,0.90,0.8,0.7,0.75,0.6,0.5,0.4,0.3,0.25,0.2,0.1,0.01]     # quali percentili calcolare
min_p=1            # percentile della banda minimo per il test
max_p=99           # percentile della banda massimo per il test

In [4]:
def simulate_model(nome,rolling,n_sim,perc, min_p,max_p):
    import pandas as pd
    import numpy as np
    import random
    from scipy.stats import norm
    def load_df(nome):
        df=pd.read_csv(nome, sep=',') #leggo file
        df.index=df['Date']         #Prendo Date come indice
        df=df.drop('Date',axis=1)   #quindi tolgo la colonna Date
        return df

    def calc_yield(df,var='Adj Close'):      # rendimento
        df['yield']=df[var].pct_change()
        return df
    
    def mobile_stdev(df, var,rolling):       # dev.standard mobile
        df2=df.copy()
        df2['mob_std'] = df[var].rolling(rolling).std().fillna(0)
        return df2
    
    def simulate(df, n_sim, rolling):   # simulazione del prezzo tramite monte carlo
        var='Adj Close'
        df2=df.copy()
        df2['mob_std'] = df[var].rolling(rolling).std().fillna(0)
        df2= df2.drop([var,'yield'],axis=1)
        df1=pd.DataFrame(index=list(df.transpose().columns)).transpose()
        for k in range(0,n_sim):
            df3 =pd.DataFrame((df2['mob_std'].values*norm.ppf(random.uniform(0, 1),scale=1))+df[var].values,index=list(df.transpose().columns)).transpose()
            df1 = df1.append(df3).reset_index(drop=True)
        return df1.reset_index(drop=True)
    
    
    def percentili(df,perc=[0.95,0.90,0.8,0.7,0.75,0.6,0.5,0.4,0.3,0.25,0.2,0.1,0.01]):  # calcolo del VAR ValueAtRisk
        df2=pd.DataFrame(df.quantile(0.99,axis=0)).transpose()
        df2.index=['perc_'+ str(int(0.99*100))]
        for k in perc:
            df3=pd.DataFrame(df.quantile(k,axis=0)).transpose()
            df3.index=['perc_'+ str(int(k*100))]
            df2=df2.append(df3)
        return df2

    def bande(df2,min_p,max_p): # modello finanziario, determina l'apertura o chiusura di una posizione se il prezzo rompa le bande calcolate
        df=df2.copy()
        df['rottura_bande'] = (df['percentili'] - df['price']['Adj Close'].transpose()).transpose()[['perc_'+str(min_p),'perc_'+str(max_p)]]
        df['rottura_bande']['perc_'+str(max_p)]= np.where(df['rottura_bande']['perc_'+str(max_p)]<0, 1, 0)
        df['rottura_bande']['perc_'+str(min_p)]= np.where(df['rottura_bande']['perc_'+str(min_p)]>0, 1, 0)
        df['rottura_bande']['rottura']=df['rottura_bande']['perc_'+str(max_p)]+df['rottura_bande']['perc_'+str(min_p)]
        df['rottura_bande']['test_rottura_bande'] = np.where(df['rottura_bande']['rottura']==0, 1, 0)
        df['rottura_bande_max'] = (df['percentili'] - df['price']['High'].transpose()).transpose()[['perc_'+str(max_p)]]
        df['rottura_bande_max']['test>max']= np.where(df['rottura_bande_max']['perc_'+str(max_p)]<0, 0, 1)
        df['rottura_bande_min'] = (df['percentili'] - df['price']['Low'].transpose()).transpose()[['perc_'+str(min_p)]]
        df['rottura_bande_min']['test<min']= np.where(df['rottura_bande_min']['perc_'+str(min_p)]>0, 0, 1)
        df['rottura_bande_min']


        df['bande']=df['price'][['Low','Adj Close','High','yield']].join(df['percentili'].transpose()[['perc_'+str(min_p),'perc_50','perc_'+str(max_p)]], lsuffix='_caller', rsuffix='_other')
        df['bande']=df['bande'].join(df['rottura_bande_max']['test>max'], lsuffix='_caller', rsuffix='_other')
        df['bande']=df['bande'].join(df['rottura_bande_min']['test<min'], lsuffix='_caller', rsuffix='_other')
        df['bande'] = df['bande'].join(df['rottura_bande']['test_rottura_bande'], lsuffix='_caller', rsuffix='_other')
        df['bande'] = df['bande'][['yield','perc_'+str(min_p),'Low','Adj Close','perc_50','High','perc_'+str(max_p), 'test<min','test>max','test_rottura_bande']]
        df['bande']['bandwith']= (df['bande']['perc_'+str(max_p)]-df['bande']['perc_'+str(min_p)])/df['bande']['perc_50']
        #df['bande']['divisore'] = (df['bande']['perc_'+str(max_p)]-df['bande']['perc_'+str(min_p)])
        #df['bande']['b']= (df['bande']['Adj Close']-df['bande']['perc_'+str(min_p)])/df['bande']['divisore']

        return df['bande']
    
    def result(df):               # calcola statistiche sul risultato
        a=df['test_rottura_bande'].sum()/len(df)
        b=df['test<min'].sum()/len(df)
        c=df['test>max'].sum()/len(df)
        df_out=pd.DataFrame([a,b,c],index=['test_rottura_bande','test<min','test>max'],columns=['OK']).transpose()
        return df_out
    
    def save(df,nome):        # lo salva come file xlsx
        nome=nome.replace('.csv','out.xlsx')
        with pd.ExcelWriter(nome) as writer:  
            df['mob_std'].to_excel(writer, sheet_name='price')
            df['simulate'].to_excel(writer, sheet_name='simulate')
            df['percentili'].to_excel(writer, sheet_name='percentile')
            df['bande'].to_excel(writer, sheet_name='bande')
            df['risultato'].to_excel(writer, sheet_name='result')





    
    df={}
    df['price'] = load_df(nome)
    df['price']=calc_yield(df['price'])
    df['mob_std']=mobile_stdev(df['price'],var = 'Adj Close', rolling=rolling)
    print('Step1: Simulazioni')
    df['simulate']=simulate(df['price'],n_sim,rolling)
    print('Step2: Percentili')
    df['percentili']=percentili(df['simulate'],perc=perc)
    print('Step3: Bande e risultato finale')
    df['bande'] = bande(df,min_p=min_p,max_p=max_p)
    df['risultato']=result(df['bande'])
    print('Step4: Salvataggio')
    try:
        save(df,nome)
        print('Fatto')
    except:
        print('Chiudere il file excel, probabilmente è aperto!')
    


simulate_model(nome,rolling,n_sim,perc, min_p,max_p)   #esecuzione del modello

Step1: Simulazioni
Step2: Percentili
Step3: Bande e risultato finale
Step4: Salvataggio
DONE
